In [1]:
import hashlib
import struct
from typing import List, Dict, Tuple

class KaoruBridge:
    """
    Forensic decomposition of SHA-256 block structures.
    Extracts W-schedule values from Bitcoin block headers.
    """

    @staticmethod
    def construct_block_header(version: int, prev_hash: bytes, merkle_root: bytes, timestamp: int, bits: int, nonce: int) -> bytes:
        """Construct the 80-byte block header."""
        return (struct.pack('<I', version) +
                prev_hash +
                merkle_root +
                struct.pack('<I', timestamp) +
                struct.pack('<I', bits) +
                struct.pack('<I', nonce))

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        """Bitcoin's double SHA-256."""
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def extract_w_values(block: bytes) -> List[int]:
        """Extract first 16 W-values from message block."""
        return [struct.unpack('>I', block[i:i+4])[0] for i in range(0, 64, 4)]

    @classmethod
    def verify_block(cls, version: int, prev_hash: bytes, merkle_root: bytes, timestamp: int, bits: int, nonce: int, expected_hash: str) -> Tuple[bool, Dict]:
        """Verify a block header and extract W-schedule values."""
        header = cls.construct_block_header(version, prev_hash, merkle_root, timestamp, bits, nonce)
        hash_result = cls.double_sha256(header)[::-1].hex()
        w_values = cls.extract_w_values(header[:64])

        return hash_result == expected_hash, {
            'header_hex': header.hex(),
            'computed_hash': hash_result,
            'expected_hash': expected_hash,
            'w_values': [hex(w) for w in w_values],
            'verification': hash_result == expected_hash
        }

# Ejemplo de uso con el bloque génesis de Bitcoin
if __name__ == "__main__":
    version = 1
    prev_hash = bytes(32)  # All zeros for the genesis block
    merkle_root = bytes.fromhex('4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b')[::-1]
    timestamp = 1231006505
    bits = 486604799
    nonce = 2083236893
    expected_hash = "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"

    success, data = KaoruBridge.verify_block(version, prev_hash, merkle_root, timestamp, bits, nonce, expected_hash)
    print(f"Verification: {'PASSED' if success else 'FAILED'}")
    print(f"Hash: {data['computed_hash']}")
    print(f"Expected: {data['expected_hash']}")
    print(f"W-Values: {data['w_values']}")

Verification: PASSED
Hash: 000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f
Expected: 000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f
W-Values: ['0x1000000', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x3ba3edfd', '0x7a7b12b2', '0x7ac72c3e', '0x67768f61', '0x7fc81bc3', '0x888a5132', '0x3a9fb8aa']
